## Import and Path Stuff

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense,GlobalAveragePooling2D
)
from tensorflow.keras import backend as K
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.models import load_model

In [3]:
def convert_image_to_array(image_dir):
    image = cv2.imread(image_dir)
    if image is not None :
        image = cv2.resize(image, default_image_size)   
        return img_to_array(image)
    else :
        return np.array([])

In [4]:
with open("/content/drive/MyDrive/Datasets/PlantVillage-Dataset-master/raw/latest/label_list.txt", "rb") as fp:
  label_list = pickle.load(fp)
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)

In [5]:
n_classes = len(label_binarizer.classes_)

In [6]:
x_train = np.load('/content/drive/MyDrive/Datasets/PlantVillage-Dataset-master/raw/latest/x_train.npy')
x_test = np.load('/content/drive/MyDrive/Datasets/PlantVillage-Dataset-master/raw/latest/x_test.npy')
y_train = np.load('/content/drive/MyDrive/Datasets/PlantVillage-Dataset-master/raw/latest/y_train.npy')
y_test = np.load('/content/drive/MyDrive/Datasets/PlantVillage-Dataset-master/raw/latest/y_test.npy')

In [7]:
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
  )

In [8]:
model = Sequential()
inputShape = (256, 256, 3)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 activation (Activation)     (None, 256, 256, 32)      0         
                                                                 
 batch_normalization (BatchN  (None, 256, 256, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 85, 85, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 85, 85, 64)        18496     
                                                                 
 activation_1 (Activation)   (None, 85, 85, 64)        0

In [10]:
opt = Adam(learning_rate=1e-5, decay=1e-5 / 25)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

In [12]:
# history = model.fit_generator(
#     aug.flow(x_train, y_train, batch_size=25),
#     validation_data=(x_test, y_test),
#     steps_per_epoch=len(x_train) // 25,
#     epochs=EPOCHS, verbose=1
#     )

In [13]:
# model.save('/content/drive/MyDrive/Datasets/PlantVillage-Dataset-master/raw/latest/base_model.h5')

In [14]:
# print(history.history)

In [15]:
# pickle.dump(history,open('/content/drive/MyDrive/Datasets/PlantVillage-Dataset-master/raw/latest/history.pkl', 'wb'))

## Load models

In [16]:
model = load_model('/content/drive/MyDrive/Datasets/PlantVillage-Dataset-master/raw/latest/base_model.h5')

In [24]:
print("[INFO] Calculating model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

[INFO] Calculating model accuracy
48/48 [==============================] - 25s 66ms/step - loss: 0.0775 - accuracy: 0.9168
Test Accuracy: 91.67999999999999


In [18]:
classes = list(label_binarizer.classes_)

In [19]:
def get_predictions(model, test_input_val):
    test_input_val = np.reshape(test_input_val, [1, 256, 256, 3])
    preds = model(test_input_val)
    preds = np.round(np.clip(preds,0,1))
    preds=preds.astype(int) 
    preds = preds[0]
    return preds

In [20]:
def getclassfrompred(pred):
  for i in range(len(pred)):
    if pred[i]==1:
      return classes[i]

In [21]:
print(getclassfrompred(get_predictions(model, x_train[78])))

Corn_(maize)___Common_rust_
